<a href="https://colab.research.google.com/github/tiagosilveiraa/portifoliods/blob/main/Recomenda%C3%A7%C3%A3o_de_Jogadores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1 - Parametrização



In [ ]:
#@title
#Capturo as principais ligas exibidas na tela principal
all_leagues = requests.get('https://api.sofascore.com/api/v1/config/unique-tournaments/BR/football').json()['uniqueTournaments']
#Variaveis que terão os dados tratados para o dropdown
dp_leagues_data = [] 
dp_seasons_data = []
#Montagem das opções dos dropdowns
for league in all_leagues:  
  dp_leagues_data.append((league['name'], league['id']))
  league_seasons = requests.get(f'https://api.sofascore.com/api/v1/unique-tournament/{league["id"]}/seasons').json()['seasons'] 
  for season in league_seasons:
    dp_seasons_data.append((season['year'], season['id'], league['id']))      

No menu abaixo selecione as ligas e temporadas que deseja obter os dados.

In [5]:
#@title
#UI
add_button = widgets.Button(description="Adicionar")
clear_button = widgets.Button(description="Limpar Seleções")
dp_leagues = widgets.Dropdown(options=dp_leagues_data, value=None)
dp_seasons = widgets.Dropdown(value=None)
UI_INPUTS = widgets.HBox([dp_leagues, dp_seasons])
UI_BUTTONS = widgets.HBox([add_button, clear_button])
output = widgets.Output()
selected_leagues = []
selected_seasons = []

def on_change(event):
  dp_seasons_league = []
  if event['name'] == 'value':
    for ds in dp_seasons_data:
      #Busco apenas pelas seasons da liga selecionada pela posição na tupla
      if ds[2] == event['new']: 
        dp_seasons_league.append(ds[0:2])
    dp_seasons.options = dp_seasons_league

def add_league(b):
  if dp_leagues.value == None or dp_seasons.value == None:
    output.clear_output()
    print('Informações não preenchidas!')
  elif dp_leagues.value in selected_leagues and dp_seasons.value in selected_seasons:
    with output:      
      output.clear_output()
      print(f"Erro: Liga e temporada já adicionadas")       
  else:
    selected_leagues.append(dp_leagues.value)
    selected_seasons.append(dp_seasons.value) 
    output.clear_output()   
    with output:      
      print('Liga adicionada com sucesso! :D')

def clear_selection(b):
  selected_leagues.clear()
  selected_seasons.clear()
  with output:      
      print('Parâmetros reiniciados.')
      
dp_leagues.observe(on_change)
add_button.on_click(add_league)
clear_button.on_click(clear_selection)
display(UI_INPUTS, UI_BUTTONS, output)

Output()

Agora selecione as posições dos jogadores que terão seus dados extraídos.

*Para selecionar mais de uma posição use as teclas CRTL ou SHIFT*

In [6]:
#@title
dp_positions = widgets.SelectMultiple(
    options=[('Goleiros', 'G'), ('Defensores', 'D'), ('Meio-Campistas', 'M'), ('Atacantes','F')],    
    value=['G', 'D', 'M', 'F'],
    #rows=10,
    description='Posições: ',
    disabled=False
)
display(dp_positions)

SelectMultiple(description='Posições: ', index=(0, 1, 2, 3), options=(('Goleiros', 'G'), ('Defensores', 'D'), …

In [232]:
import numpy as np
import pandas as pd
import requests
import asyncio
from datetime import date
from ipywidgets import widgets, interact
from IPython.display import display
import seaborn as sns
from matplotlib import rcParams
import matplotlib.pyplot as plt
from google.colab import data_table
from scipy import stats

In [249]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 100)
rcParams['figure.figsize'] = 20,9.27
rcParams['axes.titlesize'] = 22
rcParams['axes.labelsize'] = 16
#O DataTable está com problemas com o interact
data_table.disable_dataframe_formatter()
np.random.seed(777)

#2 - Obtenção das estatisticas
Nesta etapa iremos utilizar os parâmetros definidos acima para capturar os dados no SofaScore

In [7]:
#@title
def make_url_stats(league_id, season_id, position, offset):
  return f"https://api.sofascore.com/api/v1/unique-tournament/{league_id}/season/{season_id}/statistics?offset={offset}&order=-rating&accumulation=total&filters=position.in.{position}&fields=%2CbigChancesMissed%2CsuccessfulDribbles%2CsuccessfulDribblesPercentage%2CtotalShots%2CshotsOnTarget%2CshotsOffTarget%2CblockedShots%2CgoalConversionPercentage%2CpenaltiesTaken%2CpenaltyGoals%2CpenaltyWon%2CshotFromSetPiece%2CfreeKickGoal%2CgoalsFromInsideTheBox%2CgoalsFromOutsideTheBox%2CheadedGoals%2CleftFootGoals%2CrightFootGoal%2CshitWoodwork%2Coffsides%2CpenaltyConversion%2CsetPieceConversion%2Crating%2Cinterceptions%2CpenaltyConceded%2Cclearances%2CerrorLeadToGoal%2CerrorLeadToShot%2CownGoals%2CdribbledPast%2CcleanSheet%2C rating %2CbigChancesCreated%2Cassists %2CaccuratePasses%2CinaccuratePasses%2CtotalPasses%2CaccuratePassesPercentage %2CaccurateOwnHalfPasses%2CaccurateOppositionHalfPasses%2CaccurateFinalThirdPasses%2CkeyPasses%2CaccurateCrosses%2CaccurateCrossesPercentage%2CaccurateLongBalls%2CaccurateLongBallsPercentage%2CpassToAssist%2Crating %2CcleanSheet%2CpenaltyFaced%2CpenaltySave%2CsavedShotsFromInsideTheBox%2CsavedShotsFromOutsideTheBox%2CgoalsConcededInsideTheBox%2CgoalsConcededOutsideTheBox%2Cpunches%2CsuccessfulRunsOut%2ChighClaims%2CcrossesNotClaimed%2Crating %2CyellowCards%2CredCards%2CgroundDuelsWon%2CgroundDuelsWonPercentage%2CaerialDuelsWon%2CaerialDuelsWonPercentage%2CtotalDuelsWon%2CtotalDuelsWonPercentage%2CminutesPlayed%2CwasFouled%2Cfouls%2Cdispossessed%2CpossessionLost%2Cappearances%2CmatchesStarted%2Crating &limit=20"

In [8]:
#@title
def get_stats(leagues_ids, seasons_ids, positions_codes):
  players_data = list()
  for idx, league_id in enumerate(leagues_ids):       
    for position in positions_codes:      
      offset = 0
      while True:                
        #Efetuo a requisição a API, como as seasons estão ordenadas, passo apenas o indice atual
        response = requests.get(make_url_stats(league_id, seasons_ids[idx], position, offset))
        #Já percoreu todos os players então paro
        if response.json()['results'] == []:        
           break;        
        #Insiro o ID e nome da Liga no Dicionário     
        response = response.json()['results']
        for player in response:
          player['leagueId'] = league_id
          player['leagueName'] = [lgn['name'] for lgn in all_leagues if lgn['id'] == league_id][0]                       
        #Concateno os novos dados com a lista já existente e incremento o offset para a próxima requisição
        players_data.extend(response) 
        offset = offset + 20 
  return players_data

In [132]:
#@title
json_data = get_stats(selected_leagues, selected_seasons, dp_positions.value)
df_players = pd.json_normalize(json_data, max_level=6)       

#3 - Enriquecimento dos dados
Na etapa anterior obtivemos todas as estatisticas disponíveis para os jogadores, agora vamos fazer o enriquecimento desses dados com outras informações.

Buscando por posições alternativas para cada player, bem como algumas caracteristicas, como altura, valor de mercado e etc...

In [10]:
async def parallel_requests(coroutines):
  await asyncio.gather(*coroutines)

In [18]:
players_atributtes = []
players_alternative_positions = []

async def get_player_attributes(player_id):
  response_data = requests.get(f'https://api.sofascore.com/api/v1/player/{player_id}')
  response = response_data.json()['player']
  response['player.id'] = player_id #Para fazer o merge depois
  players_atributtes.append(response)

async def get_alternative_positions(player_id):
  response_data = requests.get(f'https://api.sofascore.com/api/v1/player/{player_id}/characteristics')
  response = response_data.json()
  if len(response['positions']) >=1:
    response = {key: response[key] for key in response.keys() if key == 'positions'}
    response['player.id'] = player_id #Para fazer o merge depois
    players_alternative_positions.append(response)

In [20]:
req_attributes = [get_player_attributes(p) for p in df_players['player.id']]
req_alternative_positions = [get_alternative_positions(p) for p in df_players['player.id']]

In [21]:
await parallel_requests(req_alternative_positions)

In [22]:
await parallel_requests(req_attributes)

In [133]:
df_players_attributes = pd.json_normalize(players_atributtes, max_level=2)
df_players_alternative_positions = pd.json_normalize(players_alternative_positions, max_level=2)

In [134]:
df_unified_extras = df_players_alternative_positions.merge(df_players_attributes, on='player.id')

In [135]:
#Como vieram muitas informações redundantes iremos dropá-las
df_unified_extras = df_unified_extras.drop(columns=['name', 'firstName', 'lastName', 'slug', 'shortName','proposedMarketValue', 'jerseyNumber','team.slug', 'team.shortName', 'team.gender', 'team.sport.name',
       'team.sport.slug', 'team.sport.id', 'team.tournament.name',
       'team.tournament.slug', 'team.tournament.category',
       'team.tournament.uniqueTournament', 'team.tournament.priority',
       'team.tournament.id', 'team.primaryUniqueTournament.name',
       'team.primaryUniqueTournament.slug',
       'team.primaryUniqueTournament.category',
       'team.primaryUniqueTournament.userCount',
       'team.primaryUniqueTournament.id',
       'team.primaryUniqueTournament.displayInverseHomeAwayTeams',
       'team.userCount', 'team.nameCode', 'team.disabled', 'team.national',
       'team.type', 'team.id', 'team.country.alpha2', 'team.country.name',
       'team.teamColors.primary', 'team.teamColors.secondary',
       'team.teamColors.text', 'userCount', 'id', 'team.name'])

In [136]:
df_players = df_unified_extras.merge(df_players, on='player.id')

## 3.2 - Engenharia reversa em features percentuais

Para alguns fundamentos é exibida a quantidade e o percentual de acertos do jogador, mas não há a infomação de quantos quandos lances ele errou. 

Para o nosso modelo é mais interessante termos os dados dos acertos e dos erros do que os percentuais, por isso usei a regra de 3 para criar novas features demonstrando os erros cometidos em cada fundamento

In [137]:
def reverse_feature(frame, number, percentage, method = np.ceil):
 return ((frame[number] / (frame[percentage]/100)) - frame[number]).fillna(0).apply(lambda x: int(method(x)))

In [138]:
df_players['aerialDuelsLost'] = reverse_feature(df_players, 'aerialDuelsWon', 
                                                'aerialDuelsWonPercentage')

In [139]:
df_players['groundDuelsLost'] = reverse_feature(df_players, 'groundDuelsWon', 
                                              'groundDuelsWonPercentage')

In [140]:
df_players['unsuccessfulDribbles'] = reverse_feature(df_players, 'successfulDribbles', 
                                                     'successfulDribblesPercentage', np.ceil)

In [141]:
df_players['unnacurateLongBalls'] = reverse_feature(df_players, 'accurateLongBalls',
                                                    'accurateLongBallsPercentage')

In [142]:
df_players['unaccurateCrosses'] = reverse_feature(df_players, 'accurateCrosses', 
                                                            'accurateCrossesPercentage')

In [143]:
df_players['successfulShotFromSetPiece'] = (df_players['shotFromSetPiece'] * (df_players['setPieceConversion']/100)).apply(lambda x: int(np.ceil(x)))

In [144]:
df_players['unsuccessfulShotFromSetPiece'] = reverse_feature(df_players, 'successfulShotFromSetPiece',
                                                             'setPieceConversion', np.floor)

#4 - Tratamento dos dados

In [145]:
df_players.rename(columns={'country.alpha2':'countryCode', 
                'country.name':'countryName', 
                'proposedMarketValueRaw.value':'proposedMarketValue', 
                'proposedMarketValueRaw.currency': 'proposedMarketCurrency',
                'player.id': 'playerId', 
                'player.name': 'playerName', 
                'player.slug': 'playerSlug',
                'player.userCount': 'playerUserCount',
                'team.id':'teamId',
                'team.name':'teamName',
                'team.slug':'teamSlug', 
                'team.teamColors.primary': 'teamPrimaryColor',
                'team.teamColors.secondary': 'teamSecondaryColor',
                'team.userCount': 'teamUserCount', 
                'position': 'playerMainPosition', 
                'positions': 'playerAlternativePositions'
                }, inplace=True)

In [146]:
df_players.drop(['team.teamColors.text', 
'team.type',
'team.sport.id',
'team.sport.name',
'team.sport.slug',
'team.shortName'], axis=1, inplace=True)

In [147]:
df_players = df_players[df_players.columns.sort_values(ascending=True)]

In [148]:
#Só quero players que estiveram em alguma partida na temporada
df_players = df_players[df_players['appearances']>0]

##4.1 - Tratando valores nulos

In [278]:
df_players.columns[df_players.isna().any()].tolist()

[]

Para o prazo de término dos contratos dos jogadores iremos assumir que quem não possuir esta informação preenchida irá ter o seu contrato encerrado no fim da próxima temporada.

In [150]:
from datetime import date
import time
next_year = date.today().year + 1
next_season_end = date(next_year, 12,31)
next_season_end = time.mktime(next_season_end.timetuple())

In [151]:
df_players.fillna({'height': np.ceil(df_players['height'].mean()),
                   'preferredFoot': df_players['preferredFoot'].mode().values[0], 
                   'shirtNumber': 0,
                   'retired': False,
                   'proposedMarketCurrency': 'EUR',
                   'proposedMarketValue': 0,
                   'contractUntilTimestamp': next_season_end                   
                   }, inplace=True)

O valor de mercado de um jogador é imprescindivel para a comparação de similaridade, dessa forma, não podem estar zerados.

Além disto, não posso preenchê-lo puramente com a média pois os valores por posição e liga podem variar, por isso efetuo este tratamento abaixo.

In [152]:
position_and_league_of_players_without_value = df_players[df_players['proposedMarketValue'] ==0][['playerMainPosition', 'leagueId']]
for pl in position_and_league_of_players_without_value.values:
  mean_per_position_and_league = int(df_players[(df_players['playerMainPosition'] == pl[0]) & (df_players['leagueId'] == pl[1])][['proposedMarketValue']].mean())
  df_players.loc[(df_players['proposedMarketValue'] == 0) & (df_players['playerMainPosition'] == pl[0]) & (df_players['leagueId'] == pl[1]),['proposedMarketValue']] = mean_per_position_and_league

##4.2 - Tratamento de transferências
Como é possível capturar dados de ligas diferentes, pode ser que o jogador que estava em uma Liga A tenha se transferido para uma Liga B, com isso o seu registro é exibido duas vezes o que irá causar um problema na execução do algoritmo KNN já que em breve iremos indexar o dataframe pelo ID do Jogador. Portanto, optei por eliminar os jogadores duplicados já que não faz tanto sentido comparar ele com ele mesmo.

In [153]:
df_players = df_players.drop_duplicates(subset=['playerId'], keep='last', ignore_index=True)

## 4.3 - Tratamento de Outliers [*TODO*]

In [ ]:
df_players.describe()

,accurateCrosses,accurateCrossesPercentage,accurateFinalThirdPasses,accurateLongBalls,accurateLongBallsPercentage,accurateOppositionHalfPasses,accurateOwnHalfPasses,accuratePasses,aerialDuelsLost,aerialDuelsWon,aerialDuelsWonPercentage,appearances,bigChancesCreated,bigChancesMissed,blockedShots,cleanSheet,clearances,contractUntilTimestamp,crossesNotClaimed,dateOfBirthTimestamp,dispossessed,dribbledPast,errorLeadToGoal,errorLeadToShot,fouls,freeKickGoal,goalConversionPercentage,goalsConcededInsideTheBox,goalsConcededOutsideTheBox,goalsFromInsideTheBox,goalsFromOutsideTheBox,groundDuelsLost,groundDuelsWon,groundDuelsWonPercentage,headedGoals,height,highClaims,inaccuratePasses,interceptions,keyPasses,leagueId,leftFootGoals,matchesStarted,minutesPlayed,offsides,ownGoals,passToAssist,penaltiesTaken,penaltyConceded,penaltyConversion,penaltyFaced,penaltyGoals,penaltySave,penaltyWon,playerId,playerUserCount,possessionLost,proposedMarketValue,punches,rating,redCards,savedShotsFromInsideTheBox,savedShotsFromOutsideTheBox,setPieceConversion,shirtNumber,shotFromSetPiece,shotsOffTarget,shotsOnTarget,successfulDribbles,successfulDribblesPercentage,successfulRunsOut,successfulShotFromSetPiece,teamId,teamUserCount,totalDuelsWon,totalDuelsWonPercentage,totalPasses,totalShots,unaccurateCrosses,unnacurateLongBalls,unsuccessfulDribbles,unsuccessfulShotFromSetPiece,wasFouled,yellowCards
count,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000,2.620000e+02,262.0,2.620000e+02,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000,262.0,262.000000,262.000000,262.000000,262.0,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000,262.0,262.000000,262.0,262.000000,2.620000e+02,262.000000,262.000000,2.620000e+02,262.000000,262.000000,262.000000,262.0,262.0,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000,262.0,262.000000,262.000000,262.0,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000
mean,6.129771,16.564084,125.118321,27.099237,55.927176,261.045802,175.209924,430.125954,14.675573,11.316794,40.771069,18.687023,1.652672,0.950382,4.717557,1.801527,9.500000,1.710851e+09,0.0,8.135668e+08,13.232824,16.442748,0.053435,0.087786,19.992366,0.038168,5.994427,11.255725,2.118321,1.083969,0.293893,59.740458,57.881679,47.585611,0.194656,176.702290,0.0,82.522901,12.793893,15.015267,325.0,0.400763,12.507634,1111.324427,1.286260,0.011450,0.049618,0.179389,0.103053,5.916031,0.0,0.129771,0.0,0.137405,7.228850e+05,641.709924,155.591603,2.424374e+06,0.003817,6.770840,0.148855,0.0,0.0,1.484847,21.099237,0.912214,7.450382,5.713740,15.969466,58.846718,0.0,0.041985,5487.572519,0.0,69.198473,46.605802,512.648855,17.881679,21.114504,16.587786,11.183206,0.240458,20.538168,2.706107
std,9.237519,15.525129,103.444088,28.961291,21.196535,211.088635,170.433653,362.623407,13.764498,12.454777,21.960205,10.355064,2.116703,1.583385,5.053393,2.672178,11.600221,3.960091e+07,0.0,1.371191e+08,12.151155,14.048989,0.241736,0.333224,16.194458,0.191968,10.739543,8.205658,2.050448,1.897925,0.679109,46.923446,47.628404,11.430143,0.467620,5.128531,0.0,63.303361,12.184858,14.769485,0.0,1.179194,9.951893,827.142465,2.072933,0.106596,0.217571,0.696375,0.328806,22.929087,0.0,0.617707,0.0,0.467542,3.163507e+05,1367.067690,124.254671,3.303408e+06,0.061780,0.207398,0.397283,0.0,0.0,9.991248,18.128260,2.210200,7.522853,6.391904,17.103220,23.247556,0.0,0.219178,9794.510367,0.0,55.265354,10.964027,418.898271,17.886545,29.218726,15.919274,14.170717,2.001772,19.256200,2.449779
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0000

#5 - Análise exploratória  [*Em andamento*]



##5.1 - Destaques por fundamentos

Aqui uso os outliers ao meu favor buscando por números que fogem ao desvio padrão dessa forma conseguimos identificar os players que se destacam em cada fundamento.

Caso não ache nenhum valor que se destaque, exibo os top 5.

In [246]:
#@title
features = df_players.select_dtypes(include=np.number).drop(['teamId',
'teamUserCount',
'playerUserCount',
'contractUntilTimestamp',
'dateOfBirthTimestamp'], axis=1).columns

def on_change_feature(Fundamento):
  columns_to_show = ['playerName', 'teamName', Fundamento]
  data_to_show = df_players[(np.abs(stats.zscore(df_players[Fundamento])>3))][columns_to_show].sort_values(by=Fundamento, ascending=False)
 
  if data_to_show.empty is True:    
    data_to_show = df_players.nlargest(n=5, columns=Fundamento)[columns_to_show]
  return df_players.nlargest(n=5, columns=Fundamento)[columns_to_show]


interact(on_change_feature, Fundamento=features);

interactive(children=(Dropdown(description='Fundamento', options=('accurateCrosses', 'accurateCrossesPercentag…

#6 - Geração do Modelo

In [346]:
X = df_players.copy()

##6.1 - Transformando as posições alternativas em colunas

In [347]:
X[['AM', 'DC', 'DL', 'DM', 'DR', 'LW', 'MC', 'ML', 'MR', 'RW', 'ST']] = 0

In [348]:
for i, p in enumerate(X['playerAlternativePositions'].tolist()):  
    X.loc[i, p] = 1

In [349]:
#Alterando o indice para o player.id
X.set_index('playerId', inplace = True)

##6.2 - Enconding das features categoricas

In [350]:
from sklearn.preprocessing import OneHotEncoder
f_encoder = OneHotEncoder()
c_encoder = OneHotEncoder()
feet = f_encoder.fit_transform(X[['preferredFoot']])
country = c_encoder.fit_transform(X[['countryName']])

In [351]:
X[f_encoder.categories_[0]] = feet.toarray()
X[c_encoder.categories_[0]] = country.toarray()

Seleciono apenas as colunas numéricas e dropo as de percentual pois já fizemos o tratamento das mesmas no item 3.2.

In [352]:
X = X.select_dtypes(include=np.number).drop(['goalConversionPercentage',
'penaltyConversion',
'setPieceConversion',
'accurateCrossesPercentage',
'accurateLongBallsPercentage',
'aerialDuelsWonPercentage',
'groundDuelsWonPercentage',
'successfulDribblesPercentage',
'totalDuelsWonPercentage',
'teamId',
'teamUserCount',
'playerUserCount'], axis=1)

In [353]:
from scipy.sparse import csr_matrix
stats_sparse = csr_matrix(X)

In [354]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm='brute', n_neighbors=5)
model.fit(stats_sparse)

NearestNeighbors(algorithm='brute')

# 7 - Apresentação dos Resultados

In [355]:
dp_players = df_players[['playerName','playerId']].sort_values(by=['playerName']).values.tolist()

In [356]:
def get_similars(Player):
  distance, similar_players_index = model.kneighbors(X.loc[Player].values.reshape(1,-1))
  similar_players = pd.DataFrame()
  similar_players = df_players.loc[df_players.index[similar_players_index[0]]]
  similar_players['distances'] = distance[0]
  return similar_players[['playerName', 'teamName', 'rating']]

In [357]:
interact(get_similars, Player=dp_players);

interactive(children=(Dropdown(description='Player', options=(['Aaron Connolly', 895579], ['Abel Hernández', 4…